In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/test-data/testDatafinal.csv


In [2]:
!pip install sparkmagic
!pip install pyspark

     |████████████████████████████████| 43 kB 227 kB/s            
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
     |████████████████████████████████| 121 kB 1.6 MB/s            
     |████████████████████████████████| 122 kB 7.0 MB/s            
     |████████████████████████████████| 1.3 MB 10.1 MB/s            
  Preparing metadata (setup.py) ... - done
     |████████████████████████████████| 1.7 MB 67.2 MB/s            
  Installing build dependencies ... - \ | / - done
  Getting requirements to build wheel ... - \ | / done
  Preparing metadata (pyproject.toml) ... - \ | / done
  Created wheel for sparkmagic: filename=sparkmagic-0.19.1-py3-none-any.whl size=64701 sha256=5ba3c6916559c83111ac5ec19c1432eeca09254511d01ed6ab72d86a535a0bcb
  Stored in directory: /root/.cache/pip/wheels/77/00/d5/153197547803fa423ee3d0a7a1ff19f138cbcc385a5d11e0dd
  Created wheel for autovi

In [3]:
import pyspark
from datetime import datetime 
import numpy as np
import pandas as pd
import random
import time
import matplotlib.pyplot as plt
from itertools import chain
from pyspark.sql import Row, Column
from pyspark.sql.types import BooleanType
from pyspark.sql.functions import *
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder
from pyspark.ml.feature import StandardScaler, Imputer
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier, LinearSVC
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.sql import SparkSession

#Create PySpark SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/22 04:10:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df_flights_weather_sm = pd.read_csv('/kaggle/input/test-data/testDatafinal.csv')

In [5]:
df_flights_weather_sm['YEAR_OBS']= pd.to_datetime(df_flights_weather_sm['YEAR_OBS'])
df_flights_weather_sm['YEAR_OBS'] = pd.DatetimeIndex(df_flights_weather_sm['YEAR_OBS']).year

In [6]:
sparkDF=spark.createDataFrame(df_flights_weather_sm) 
sparkDF.printSchema()

root
 |-- NEIGHBOR_ID: long (nullable = true)
 |-- FL_DATE: string (nullable = true)
 |-- DEP_HOUR_OF: long (nullable = true)
 |-- MONTH: long (nullable = true)
 |-- YEAR: long (nullable = true)
 |-- STATION: long (nullable = true)
 |-- DATE_OBS: string (nullable = true)
 |-- FLIGHT_HOUR_OBS: long (nullable = true)
 |-- MONTH_OBS: long (nullable = true)
 |-- YEAR_OBS: long (nullable = true)
 |-- DEP_DEL15: long (nullable = true)



In [7]:
hold_out_variable = 'YEAR_OBS'
hold_out_threshold = '2019'

In [8]:
trainDF = sparkDF.filter(sparkDF[hold_out_variable]!=hold_out_threshold).cache()
testDF = sparkDF.filter(sparkDF[hold_out_variable]==hold_out_threshold).cache()

In [9]:
sparkDF=spark.createDataFrame(df_flights_weather_sm) 
sparkDF.printSchema()

root
 |-- NEIGHBOR_ID: long (nullable = true)
 |-- FL_DATE: string (nullable = true)
 |-- DEP_HOUR_OF: long (nullable = true)
 |-- MONTH: long (nullable = true)
 |-- YEAR: long (nullable = true)
 |-- STATION: long (nullable = true)
 |-- DATE_OBS: string (nullable = true)
 |-- FLIGHT_HOUR_OBS: long (nullable = true)
 |-- MONTH_OBS: long (nullable = true)
 |-- YEAR_OBS: long (nullable = true)
 |-- DEP_DEL15: long (nullable = true)



In [10]:
trainDF.groupBy('YEAR_OBS').count().show()
testDF.groupBy('YEAR_OBS').count().show()

+--------+-----+
|YEAR_OBS|count|
+--------+-----+
|    2016|  199|
|    2015|  200|
|    2017|  201|
|    2018|  200|
+--------+-----+

+--------+-----+
|YEAR_OBS|count|
+--------+-----+
|    2019|  200|
+--------+-----+



In [11]:
minor_df = trainDF.filter(col('DEP_DEL15')==1).cache()
major_df = trainDF.filter(col('DEP_DEL15')==0).cache()

In [12]:
n_ontime = major_df.count()
n_delays = minor_df.count()

In [13]:
ratio = n_ontime/n_delays
print('The ratio of on-time to delayed flights is of {:0.1f}:1'.format(ratio))

The ratio of on-time to delayed flights is of 10.3:1


In [14]:
oversample_df = minor_df.sample(withReplacement=True, fraction=ratio, seed=321)
augmentedTrainDF = major_df.unionAll(oversample_df).cache()
augmentedTrainDF.groupBy('DEP_DEL15').count().show()

+---------+-----+
|DEP_DEL15|count|
+---------+-----+
|        0|  729|
|        1|  721|
+---------+-----+



In [15]:
fold_variable = 'YEAR_OBS'
fold_list = augmentedTrainDF.select(fold_variable).distinct().toPandas()[fold_variable]
mapping = {x: x - fold_list.min() for x in fold_list}
print(f'{fold_variable.capitalize()}, fold_number mapping: {mapping}')

Year_obs, fold_number mapping: {2016: 1, 2015: 0, 2017: 2, 2018: 3}


In [16]:
nFolds = len(fold_list) - 1
print(f'Total number of folds: {nFolds}')

Total number of folds: 3


In [17]:
display(mapping)

{2016: 1, 2015: 0, 2017: 2, 2018: 3}

In [18]:
# add 'foldCol' to TrainDF
# mapping_expr = create_map([lit(x) for x in chain(*mapping.items())]
mapping_expr = create_map([lit(x) for x in chain(*mapping.items())])
foldedTrainDF = augmentedTrainDF.withColumn('foldCol', mapping_expr[col(fold_variable)]).cache()

In [19]:
class TimeSeriesCrossValidator(CrossValidator):
    '''
    Customizes CrossValidator to perform time series cross validation on a rolling basis.
    User needs to provide `foldCol` with the fold numbers defined in a time ascending order
    (e.g. 2015 is assigned as fold 0, 2016 as fold 1, and so on).
    '''
    def _kFold(self, dataset):
        nFolds = self.getOrDefault(self.numFolds)
        foldCol = self.getOrDefault(self.foldCol)

        datasets = []
        if not foldCol:
            # Do random k-fold split.
            seed = self.getOrDefault(self.seed)
            h = 1.0 / nFolds
            randCol = self.uid + "_rand"
            df = dataset.select("*", rand(seed).alias(randCol))
            for i in range(nFolds):
                validateLB = i * h
                validateUB = (i + 1) * h
                condition = (df[randCol] >= validateLB) & (df[randCol] < validateUB)
                validation = df.filter(condition)
                train = df.filter(~condition)
                datasets.append((train, validation))
        else:
            # Use user-specified fold numbers.
            def checker(foldNum):
                if foldNum < 0 or foldNum > nFolds:
                    raise ValueError(
                        "Fold number must be in range [0, %s], but got %s." % (nFolds, foldNum)
                    )
                return True

            checker_udf = UserDefinedFunction(checker, BooleanType())
            for i in range(nFolds):
                training = dataset.filter(checker_udf(dataset[foldCol]) & (col(foldCol) <= lit(i))) # Training set always in the past
                validation = dataset.filter(
                    checker_udf(dataset[foldCol]) & (col(foldCol) == lit(i+1)) # Validation set always in the future
                )
                if training.rdd.getNumPartitions() == 0 or len(training.take(1)) == 0:
                    raise ValueError("The training data at fold %s is empty." % i)
                if validation.rdd.getNumPartitions() == 0 or len(validation.take(1)) == 0:
                    raise ValueError("The validation data at fold %s is empty." % i)
                datasets.append((training, validation))

        return datasets